### Multi Layer Perceptron

In [ ]:
# Install jdc
%%bash

pip install jdc

In [2]:
# Library imports

import random
import numpy as np
import jdc

We define a generic neural network architecture as a python class which we would use in multiple exercies. You might want to revisit the tutorial notebook for a quick refresher on python classes.

**Note:** We are using jdc to define each method of `class Network` in seperate cells. jdc follows the following syntex,

```py
%%add_to #CLASS_NAME#
def dummy_method(self):
```

In [1]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network. For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.initialize_biases()
        self.initialize_weights()

The biases and weights for the network are initialized randomly, using a Gaussian
distribution with mean 0, and variance 1. Note that the first layer is assumed to be an input layer, and by convention we won't set any biases for those neurons, since biases are only ever used in computing the outputs from later layers. Implement the following functions to initialize biases and weights.

**Hints:**
![](https://www.researchgate.net/profile/Michael_Frish/publication/241347660/figure/fig3/AS:298690993508361@1448224890429/Figure-3-The-structure-of-a-multilayer-perceptron-neural-network.png)
- Since we do not define biases for input layer, `len(self.biases)` array is equal to `len(self.sizes) - 1`.
- Every consecutive pair of layers in network have a set of weights connecting them. Hence the `len(self.weights)` would also be `len(self.sizes) - 1` .
- `np.random.randn` picks random variables form gaussian distribution.

In [ ]:
%%add_to Network
def initialize_biases(self):
    self.biases = [np.random.randn(y, 1) for y in sizes[1:]]

%%add_to Network
def initialize_weights(self):
    self.weights = [np.random.randn(y, x)
                    for x, y in zip(sizes[:-1], sizes[1:])]

In [ ]:
%%add_to Network
def feedforward(self, a):
    """Return the output of the network if ``a`` is input."""
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a) + b)
    return a

In [ ]:
%%add_to Network
def train(self, training_data, epochs, learning_rate):
    """Train the neural network using gradient descent.  
    The ``training_data`` is a list of tuples
    ``(x, y)`` representing the training inputs and the desired
    outputs.  The other parameters are self-explanatory."""

    for i in xrange(epochs):       
        self.update_params(training_data, learning_rate)    
        print "Epoch {0} complete".format(i)

In [ ]:
%%add_to Network
def update_params(self, training_data, learning_rate):
    """Update the network's weights and biases by applying
    gradient descent using backpropagation."""
    delta_b = [np.zeros(b.shape) for b in self.biases]
    delta_w = [np.zeros(w.shape) for w in self.weights]
    total_cost = 0
    for x, y in training_data:
        cost, del_b, del_w = self.backprop(x, y)
        delta_b = [nb + dnb for nb, dnb in zip(delta_b, del_b)]
        delta_w = [nw + dnw for nw, dnw in zip(delta_w, del_w)]
        total_cost += cost
    print "Total cost: {}".format(total_cost)
#   Update self.biases and self.weights
    self.biases = [b - (learning_rate / len(training_data)) * db
                   for b, db in zip(self.biases, delta_b)]
    self.weights = [w - (learning_rate / len(training_data)) * dw
                    for w, dw in zip(self.weights, delta_w)]

In [ ]:
%%add_to Network
def backprop(self, x, y):
    """Return arry containiing cost, del_b, del_w representing the
    cost function C(x) and gradient for cost function.  ``del_b`` and
    ``del_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    # Forward pass
    activations, zs = forward(x)
    
    # Backward pass     
    cost, del_b, del_w = backward(activations, zs, y)

    return cost, del_b, del_w

In [ ]:
%%add_to Network
def forward(self, x):
    """Compute activation and Z for each layer and return
    arrays containing activations and Zs layer-by-layer."""
    # current activation
    activation = x
    # list to store all the activations, layer by layer
    activations = [x]
    # list to store all the activations, layer by layer
    zs = []
    
    # Loop through each layer to compute activations and Zs    
    for b, w in zip(self.biases, self.weights):
        z = np.dot(w, activation) + b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        
    return activations, zs

In [ ]:
%%add_to Network
def backward(self, activations, zs, y):
    """Compute and return cost funcation, gradients for 
    weights and biases for each layer."""
    # Initialize gradient arrays
    del_b = [np.zeros(b.shape) for b in self.biases]
    del_w = [np.zeros(w.shape) for w in self.weights]
    
    # Compute for last layer
    cost = self.mse(activations[-1], y)
    delta = self.mse_derivative(activations[-1], y) * \
        sigmoid_derivative(zs[-1])
    del_b[-1] = delta
    del_w[-1] = np.dot(delta, activations[-2].transpose())
    
    # Loop through each layer in reverse direaction to 
    # populate del_b and del_w   
    for l in xrange(2, self.num_layers):
        z = zs[-l]
        sp = sigmoid_derivative(z)
        delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
        del_b[-l] = delta
        del_w[-l] = np.dot(delta, activations[-l-1].transpose())
    
    return cost, del_b, del_w

In [ ]:
%%add_to Network
def evaluate(self, test_data):
    """Return the accuracy of Network. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation."""
    test_results = [(np.argmax(self.feedforward(x)), y)
                    for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results) / len(test_results)

In [ ]:
%%add_to Network
def mse(self, output_activations, y):
    """Returns mean square error."""
    return sum((output_activations - y) ** 2 / 2)

In [ ]:
%%add_to Network
def mse_derivative(self, output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
    \partial a for the output activations. """
    return (output_activations - y)

In [ ]:
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def sigmoid_derivative(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))